# Explode Glacier Polygons

This notebook explodes (merges) all glacier polygons that are touching to turn them into one polygon. Glaciers that are touching are part of a glacier catchment and I am interested in catchements as well as individual glaciers. Not all regions contain ice caps. The regions that have ice caps and that are processed in this notebook are the following:

* Region 3 - Arctic Canada, North
* Region 4 - Arctic Canada, South
* Region 5 - Greenland
* Region 6 - Iceland
* Region 7 - Svalbard and Jan Mayen
* Region 8 - Scandinavia
* Region 9 - Russian Arctic
* Region 10 - Asia, North
* Region 17 - Southern Andes

The ice cap and icefield outlines were created by looking at the GLIMS Viewer map (http://www.glims.org/maps/glims) and manaully selecting lat/lon bounding box coordinates around the ice caps.

Adding region 16 (low latitudes) as a region to do ice cap analysis because South America has some small ice caps, but this was not initially identified as a region that needed ice cap analysis.

Adding Region 19 sub-antartic islands - from RGI only

In [1]:
import os
import os.path as op
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

# set working dir
HOME = op.join(op.expanduser("~"))
os.chdir(os.path.join(HOME, "git/wgms-glacier-project"))

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import scripts.wgms_scripts as ws

In [2]:
# Explode the regions that need to be exploded
regions = [3, 4, 5, 6, 7, 8, 9, 10, 16, 17]
for i in regions:
    ws.explode_glaciers(i, "GLIMS")

GLIMS Region 3 has already been processed
GLIMS Region 4 has already been processed
GLIMS Region 5 has already been processed
GLIMS Region 6 has already been processed
GLIMS Region 7 has already been processed
GLIMS Region 8 has already been processed
GLIMS Region 9 has already been processed
GLIMS Region 10 has already been processed
GLIMS Region 16 has already been processed
GLIMS Region 17 has already been processed


In [3]:
# explode RGI region 7 because GLIMS is missing jan mayan
ws.explode_glaciers(7, "RGI")

RGI Region 7 has already been processed


In [5]:
# Subantartic islands - use RGI
ws.explode_glaciers(19, "RGI")

RGI 19
data/rgi/raw/19_rgi60_AntarcticSubantarctic/19_rgi60_AntarcticSubantarctic.shp


# Create Shapefile for Jan Mayan Outlines
Jan Mayan outlines were errouneously left out of the GLIMS version that I downloaded so adding them to their own exploded shapefile from the RGI data

In [ ]:
# Set output file name
jm_exploded_fn = "data/glims/processed/ice-caps/exploded/exploded_7_jan_mayan.shp"

# If the file has not already been created, create it
if os.path.exists(jm_exploded_fn) == False:
    # Open RGI exploded region 7 shapefile
    rgi_exploded_fn = "data/rgi/processed/ice-caps/exploded/exploded_7.shp"
    rgi_exploded_df = gpd.read_file(rgi_exploded_fn)

    # Create a clipping polygon for Jan Mayan
    # Create dataframe that holds the clipping box
    jan_mayan_points = Polygon([(-9.5691, 71.5205), (-7.2620, 71.5205),
                                 (-7.2620, 70.5136), (-9.5691, 70.5136), 
                                 (-9.5691, 71.5205)])
    jan_mayan_gdf = gpd.GeoDataFrame([1],
                                     geometry=[jan_mayan_points],
                                     crs={'init': 'epsg:4362'})

    # Find the RGI outlines that lie within the jan mayan outline
    jan_mayan_pip_mask = ws.pip(rgi_exploded_df, jan_mayan_gdf)

    # Pass pip_mask into data to get the ones that are in the specified region
    rgi_jan_mayan_region = rgi_exploded_df.loc[jan_mayan_pip_mask]

    # Save Jan Mayan to shapefile
    rgi_jan_mayan_region.to_file(driver='ESRI Shapefile', filename=jm_exploded_fn)
    
else:
    print("Jan Mayan file has already been processed")